In [1]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 필요한 라이브러리 설치
!pip install rdflib
!pip install sentence-transformers
!pip install torch
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:
# 온톨리지 파일에서 개념 추출
from rdflib import Graph

# 온톨로지 OWL 파일 경로
ontology_path = "/content/drive/MyDrive/문화data/Refresh.rdf"

g = Graph()
g.parse(ontology_path)

# 온톨로지 클래스(개념) 추출
concepts = []
for s, p, o in g.triples((None, None, None)):
    if p.endswith('type') and o.endswith('Class'):
        # 클래스 이름만 추출
        class_name = s.split('#')[-1]
        concepts.append(class_name)

print('온톨로지 개념:', concepts)

온톨로지 개념: ['에너지_충전', '공감됨', '공감_및_위로', '위로와_응원', '자극_받음', '활력_회복', '코믹함', '기분전환', '즐거움', '이해받는_느낌', '현실을_잊음', '신선함', '마음의_안식', '긍정적_에너지', '내_이야기를_보는_듯함', '포근함', '재미', '스트레스_해소', '다시_힘을_얻음', '시간_가는_줄_모름', '마음이_가벼워짐', '상쾌함', '위로받음', '활력_얻음', '희망_및_용기', '빠져들었음', '웃음', '정서적_안정', '몰입/집중', '기분이_좋아짐', '잔잔함', '안도감', '일상에서_벗어남', '새로움/자극', '유쾌함', '색다른_이야기', '용기_얻음', '울음/감정_해소', '평온함']


In [4]:
# 리뷰 데이터가 줄마다 한 리뷰씩 저장된 txt 파일이라고 가정
review_path = "/content/drive/MyDrive/문화data/all_watcha_reviews(top 400).txt"

with open(review_path, 'r', encoding='utf-8') as f:
    reviews = [line.strip() for line in f.readlines()]

print('리뷰 샘플:', reviews[:5])

리뷰 샘플: ['[(아는 건 별로 없지만) 가족입니다] https://pedia.watcha.com/ko-KR/contents/tPeWKYo', '“가족은 남이 찾지 못하는 급소를 너무 잘 알고 있다', '언제든 강력한 한방을 날릴 수 있다”', '', '-9화 은희 대사중']


In [5]:
# BGE-M3 임베딩 모델 로딩
from sentence_transformers import SentenceTransformer

# BGE-M3 모델명 (HuggingFace Hub 기준)
model_name = 'BAAI/bge-m3'
model = SentenceTransformer(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [6]:
# 온톨로지 개념 임베딩
concept_embeddings = model.encode(concepts, normalize_embeddings=True)

# 테스트용 리뷰 일부만 샘플링 (예: 100개)
sample_reviews = reviews[:100]
review_embeddings = model.encode(sample_reviews, normalize_embeddings=True)

In [7]:
# 코싸인 유사도 계산 및 필터링
import torch
import numpy as np

# 임계값 설정 (예: 0.5 이상)
threshold = 0.7

# 코사인 유사도 계산
filtered_reviews = []
for idx, emb in enumerate(review_embeddings):
    # 각 리뷰와 모든 온톨로지 개념 임베딩의 유사도 계산
    sims = np.dot(concept_embeddings, emb)
    if np.max(sims) >= threshold:
        filtered_reviews.append(sample_reviews[idx])

print('필터링된 리뷰:', filtered_reviews[:5])

필터링된 리뷰: []


### 전체 다시 돌리기

In [3]:
# 1. 온톨로지 파일에서 개념 추출 (rdflib 그대로 사용)
from rdflib import Graph

ontology_path = "/content/drive/MyDrive/문화data/Refresh.rdf"
g = Graph()
g.parse(ontology_path)

concepts = []
for s, p, o in g.triples((None, None, None)):
    if p.endswith('type') and o.endswith('Class'):
        class_name = s.split('#')[-1]
        concepts.append(class_name)

print('온톨로지 개념 수:', len(concepts))

온톨로지 개념 수: 39


In [4]:
# 2. 전체 리뷰 데이터 불러오기
review_path = "/content/drive/MyDrive/문화data/all_watcha_reviews_combined.txt"
with open(review_path, 'r', encoding='utf-8') as f:
    reviews = [line.strip() for line in f if line.strip()]

print('전체 리뷰 수:', len(reviews))

전체 리뷰 수: 75172


In [5]:
import re
reviews = []
with open(review_path, 'r', encoding='utf-8') as f:
    lines = [line.rstrip('\n') for line in f]
    i = 0
    while i < len(lines):
        if lines[i].startswith('['):
            # 제목 줄에서 제목만 추출
            title_match = re.match(r"\[(.*?)\]", lines[i])
            title = title_match.group(1).strip() if title_match else "제목 없음"
            i += 1
            # URL 줄은 건너뜀
            if i < len(lines) and lines[i].startswith('http'):
                i += 1
            # 리뷰 텍스트 줄(빈 줄 전까지)
            review_text = []
            while i < len(lines) and lines[i].strip() != '':
                review_text.append(lines[i])
                i += 1
            # 제목과 리뷰 텍스트만 묶어서 저장
            review_block = f"[{title}]\n" + '\n'.join(review_text)
            reviews.append(review_block)
        i += 1

print('리뷰 샘플:', reviews[:2])

리뷰 샘플: ['[(아는 건 별로 없지만) 가족입니다]\n“가족은 남이 찾지 못하는 급소를 너무 잘 알고 있다 언제든 강력한 한방을 날릴 수 있다”', '[(아는 건 별로 없지만) 가족입니다]\n첫째는 첫째같고 둘째는 둘째같다 ㅋㅋ 추자현 배우 보면서 계속 연기 잘한다고 감탄함 잘생겼음. 완벽하고 멋있다.']


In [6]:
# 3. BGE-M3 임베딩 모델 로딩
from sentence_transformers import SentenceTransformer
model_name = 'BAAI/bge-m3'
model = SentenceTransformer(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [7]:
# 4. 온톨로지 개념 임베딩
concept_embeddings = model.encode(concepts, normalize_embeddings=True)

In [8]:
import numpy as np
from tqdm import tqdm
import torch
import gc

threshold = 0.6
batch_size = 64
internal_encode_batch = 4

def extract_review_text(review_block):
    lines = review_block.split('\n')
    return '\n'.join(lines[1:]).strip()

review_texts = [extract_review_text(r) for r in reviews]

filtered_reviews = []

for i in tqdm(range(0, len(reviews), batch_size)):
    batch_texts = review_texts[i:i+batch_size]
    batch_blocks = reviews[i:i+batch_size]

    try:
        # GPU 시도
        batch_embeddings = model.encode(
            batch_texts,
            normalize_embeddings=True,
            batch_size=internal_encode_batch,
            device='cuda'
        )
    except RuntimeError as e:
        print(f"\n🚨 GPU 메모리 부족! CPU로 전환 중... ({i}~{i+batch_size})")
        torch.cuda.empty_cache()
        gc.collect()  # Python 메모리 정리

        # CPU fallback
        batch_embeddings = model.encode(
            batch_texts,
            normalize_embeddings=True,
            batch_size=internal_encode_batch,
            device='cpu'
        )

    # 유사도 계산
    sims = np.dot(batch_embeddings, concept_embeddings.T)
    for j, sim_row in enumerate(sims):
        if np.max(sim_row) >= threshold:
            filtered_reviews.append(batch_blocks[j])

    # 중간 중간 GPU 메모리 해제
    torch.cuda.empty_cache()
    gc.collect()

# 결과 확인
print('✅ 필터링된 리뷰 샘플:', filtered_reviews[:2])
print('✅ 필터링된 리뷰 수:', len(filtered_reviews))

100%|██████████| 588/588 [16:34<00:00,  1.69s/it]

✅ 필터링된 리뷰 샘플: ['[(아는 건 별로 없지만) 가족입니다]\n흔하디 흔한 자극적 전개가 아닐까싶었지만 볼수록 오묘하고 생각하게 만드는 이야기들이 흥미롭다', '[(아는 건 별로 없지만) 가족입니다]\n가족은 같이 살아온 세월이다.']
✅ 필터링된 리뷰 수: 2135


In [9]:
filtered_reviews

['[(아는 건 별로 없지만) 가족입니다]\n흔하디 흔한 자극적 전개가 아닐까싶었지만 볼수록 오묘하고 생각하게 만드는 이야기들이 흥미롭다',
 '[(아는 건 별로 없지만) 가족입니다]\n가족은 같이 살아온 세월이다.',
 '[1%의 어떤것]\n음.',
 '[18 어게인]\n뭐 유치하지만 재밌다 가볍게 보기좋다 드라마가 재밌음댔지 갑자기 교훈 +감동+ 억지 눈물만 아님 좋겠다',
 '[18 어게인]\n눈물이 났다 왠지 모르게…',
 '[18 어게인]\n기분 좋아지는 휴먼 판타지 이도현의 발견, 찐 농구 장면 인정~',
 '[20세기 소년소녀]\n잔잔할수록 깊다.',
 '[20세기 소년소녀]\n두근두근 설렌다',
 '[20세기 소년소녀]\n마음이 편안하고 따뜻해지는 그것만으로도 충분했던',
 '[365: 운명을 거스르는 1년]\n지나온 어제보다 오늘을 믿고, 알 수 없는 내일을 향해 용기를 내는 것. 주어진 오늘에 최선을 다해야 할 이유가 여기에 있다.',
 '[365: 운명을 거스르는 1년]\n참신 신선 군더더기없는 스토리...',
 '[49일]\n말로 다하지 않아도 느껴지는 진심.  서서히 스며와 나마저도 알지 못한 시간 동안 지그시 바라봐주던 따스한 마음.',
 '[7급 공무원]\n끝까지 힘을 잃지 않는 유머의 동력.',
 "[7급 공무원]\n억지 감동과 쓸데없는 진지함을 내려놓고 '성실하게' 웃긴다.",
 '[7급 공무원]\n으악 유치해',
 '[7급 공무원]\n간만에 정말 아무 생각 안하고 즐겁게 웃으며 시간 보낼 수 있었다. 자기 본연과 해야할 일에 충실했다.',
 '[7급 공무원]\n제법 많이 웃겼음. 칭찬해.',
 '[7급 공무원]\n생각없이 유쾌하게 보기엔 좋다.',
 '[7급 공무원]\n속보이는 코미디의 경쾌한 잰걸음',
 '[7급 공무원]\n억지 감동이 아닌 웃음으로만 밀어붙인 것이 인상적.',
 '[7급 공무원]\n극한직업 이전 간만에 본 수준급 코미디.',
 '[7급 공무원]\n전형적인 한국 코미디 작품.',
 '[7급 공무원]\n꺼려질 만

In [10]:
# 6. 결과 저장
filtered_path = "/content/drive/MyDrive/문화data/Ontology_filtered_reviews.txt"
with open(filtered_path, 'w', encoding='utf-8') as f:
    for review in filtered_reviews:
        f.write(review + '\n')
print('저장 완료:', filtered_path)

저장 완료: /content/drive/MyDrive/문화data/Ontology_filtered_reviews.txt
